# A Brief Introduction: Reading and Writing Files

- **Coroutine-friendly** files reading and writing are briefly introduces. 

In [1]:
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.5.2")
import kotlinx.coroutines.*
import kotlin.coroutines.CoroutineContext
import java.io.File
import java.io.IOException
import java.io.FileInputStream
import java.io.FileOutputStream
import java.io.ByteArrayOutputStream

## Getting File Full Absolute Path

In [2]:
// Creating a function to get a file's full absolute path in the working dir.

fun buildWDPath(folderName: String, fileName: String): String {
    // Use File.separator to concatenate the folder and file names
    return ".${File.separator}$folderName${File.separator}$fileName"
}

## Reading and Writing Text Files

In [3]:
// Preparing read and write functions

suspend fun writeTextFile(filePath: String, content: String): Boolean {
    return withContext(Dispatchers.IO) { // Switch to the I/O dispatcher
        try {
            // Write the provided content to the specified file
            File(filePath).writeText(content) // Writes a text
            println("Text data written successfully!") // Confirmation message
            true // Return true to indicate success
        } catch (e: IOException) {
            // Handle file write errors
            println("Error writing to file: ${e.message}") // Error message
            false // Return false to indicate failure
        }
    }
}

// Function to append text content to a file using coroutines
suspend fun appendTextFile(filePath: String, content: String): Boolean {
    return withContext(Dispatchers.IO) { // Switch to the I/O dispatcher
        try {
            // Append the provided content to the specified file
            File(filePath).appendText(content) // Appends a text
            println("Text data appended successfully!") // Confirmation message
            true // Return true to indicate success
        } catch (e: IOException) {
            // Handle file write errors
            println("Error writing to file: ${e.message}") // Error message
            false // Return false to indicate failure
        }
    }
}

// Function to read text content from a file using coroutines
suspend fun readTextFile(filePath: String): String? {
    return withContext(Dispatchers.IO) { // Switch to the I/O dispatcher
        try {
            // Read the entire content of the file as text
            val content = File(filePath).readText() // Reads as text
            println("Text data read successfully!") // Confirmation message
            content // Return the read content
        } catch (e: IOException) {
            // Handle file read errors
            println("Error reading file: ${e.message}") // Error message
            null // Return null in case of an error
        }
    }
}

In [4]:
// Writing and appending to a text file

fun main() = runBlocking {
    val folderName = "files"
    val fileName = "write-read-text.txt"
    val filePath = buildWDPath(folderName, fileName)
    val message = "Hello World: Text"

    // Writing 
    writeTextFile(filePath, message) // Does not append but replaces existing file or create a new file
    // Appending 
    appendTextFile(filePath, "\n" + message) // Appends to existing file
}

main()

Text data written successfully!
Text data appended successfully!


true

In [5]:
// Reading a text file
fun main() = runBlocking {
    val folderName = "files"
    val fileName = "write-read-text.txt"
    val filePath = buildWDPath(folderName, fileName)

    val output = readTextFile(filePath)
    println(output)
}

main()

Text data read successfully!
Hello World: Text
Hello World: Text


In [6]:
// Removing the file

fun main() = runBlocking {
    val folderName = "files"
    val fileName = "write-read-text.txt"
    val filePath = buildWDPath(folderName, fileName)
    
    withContext(Dispatchers.IO) {
        try {
            File(filePath).delete()
            println("File ${filePath} was removed.")
        } catch (e: IOException) {
            println("Error removing file: ${e.message}")
        }
    }
}

main()

File .\files\write-read-text.txt was removed.


## Reading and Writing Binary Files

In [7]:
// Preparing the function to read and write binary files

// Function to write binary data to a file using coroutines
suspend fun writeBinaryFile(filePath: String, binaryData: ByteArray): Boolean {
    return withContext(Dispatchers.IO) { // Switch to the I/O dispatcher
        try {
            // Write binary data (ByteArray) directly to the file
            File(filePath).writeBytes(binaryData) // Writes binary data
            println("Binary data written successfully!") // Confirmation message
            true // Return true to indicate success
        } catch (e: IOException) {
            // Handle any I/O errors during writing
            println("Error writing binary data to file: ${e.message}") // Error message
            false // Return false to indicate failure
        }
    }
}

// Function to read binary data from a file using coroutines
suspend fun readBinaryFile(filePath: String): ByteArray? {
    return withContext(Dispatchers.IO) { // Switch to the I/O dispatcher
        try {
            // Read binary data (ByteArray) from the file
            val data = File(filePath).readBytes() // Reads binary data
            println("Binary data read successfully!") // Confirmation message
            data // Return the read binary data
        } catch (e: IOException) {
            // Handle any I/O errors during reading
            println("Error reading binary data from file: ${e.message}") // Error message
            null // Return null in case of an error
        }
    }
}


In [8]:
// Writing data to file

fun main() = runBlocking {
    val folderName = "files"
    val fileName = "write-read-binary"
    val filePath = buildWDPath(folderName, fileName)
    val message = "Hello World: Binary"
    val binaryData = message.toByteArray(Charsets.UTF_8)

    // Writing "Hello World: Binary" as binary data
    writeBinaryFile(filePath, binaryData) 
}

main()

Binary data written successfully!


true

In [9]:
// Reading a binary file

fun main() = runBlocking {
    val folderName = "files"
    val fileName = "write-read-binary"
    val filePath = buildWDPath(folderName, fileName)
    
    val output = readBinaryFile(filePath)
    output?.let { it ->
        // Convert binary data back to text using UTF-8 encoding
        // We already know data is UTF_8 text
        val textRead = String(it, Charsets.UTF_8)
        println(textRead)
    }
}

main()

Binary data read successfully!
Hello World: Binary


In [10]:
// Removing the file

fun main() = runBlocking {
    val folderName = "files"
    val fileName = "write-read-binary"
    val filePath = buildWDPath(folderName, fileName)
    
    withContext(Dispatchers.IO) {
        try {
            File(filePath).delete()
            println("File ${filePath} was removed.")
        } catch (e: IOException) {
            println("Error removing file: ${e.message}")
        }
    }
}

main()

File .\files\write-read-binary was removed.


## Reading and Writing Files by Streams

In [11]:
// Preparing read and write by streaming functions 

// Function to read file with streaming using a buffer
suspend fun readBinaryFileWithStreaming(filePath: String): ByteArray? {
    // Switch to the IO dispatcher for executing blocking I/O operations
    return withContext(Dispatchers.IO) {
        // Initialize a buffer of 1024 bytes (1 KB) to temporarily hold data read from the file
        val buffer = ByteArray(1024)

        // Create a ByteArrayOutputStream to collect all read bytes into a single array
        val outputStream = ByteArrayOutputStream()

        try {
            // Open a FileInputStream to read the specified file
            FileInputStream(File(filePath)).use { inputStream -> // 'use' automatically closes the stream.
                /* 
                The use function is a Kotlin extension function that is called on the AutoCloseable
                object (in this case, FileInputStream). 
                It takes a lambda expression as its argument and ensures that the resource 
                is closed automatically when the block of code is exited, 
                whether it exits normally or due to an exception.
                */
                var bytesRead: Int // Variable to track the number of bytes read in each iteration

                // Continuously read data into the buffer until the end of the file is reached
                while (
                    inputStream.read(buffer).also { it -> // Read bytes into the buffer
                        bytesRead = it // Store the number of bytes read
                    } != -1 // Continue looping until -1 is returned (end of file)
                ) {
                    // Write the bytes read into the ByteArrayOutputStream
                    outputStream.write(buffer, 0, bytesRead) // Write only the number of bytes read
                }
            } // The inputStream is automatically closed after this block due to 'use'

            // Convert the collected bytes from the output stream to a ByteArray and return it
            outputStream.toByteArray()
            /*
            The ByteArrayOutputStream class is designed to write data to a byte array in memory. 
            It does not hold any system resources like file descriptors, 
            so closing it is not strictly necessary.
            */
        } catch (e: IOException) { // Catch any IOException that may occur
            // Print an error message if reading the file fails
            println("Error reading file: ${e.message}")
            // Return null to indicate that an error occurred during the read operation
            null
        }
    }
}

// Function to write content to a file using streaming
suspend fun writeBinaryFileWithStreaming(filePath: String, content: ByteArray): Boolean {
    return withContext(Dispatchers.IO) {
        try {
            FileOutputStream(File(filePath)).use { outputStream ->
                // Write the buffer (binary data) to the file
                outputStream.write(content) // Write binary data
            } // outputStream is automatically closed by 'use'
            true // Indicate successful write operation
        } catch (e: IOException) {
            println("Error writing to file: ${e.message}") // Handle any I/O errors
            false // Indicate failure
        }
    }
}


In [12]:
// Writing binary data to file with streaming

fun main() = runBlocking {
    val folderName = "files"
    val fileName = "write-read-binary-streaming"
    val filePath = buildWDPath(folderName, fileName)
    val message = "Hello World: Binary Streaming"
    val binaryData = message.toByteArray(Charsets.UTF_8)

    // Writing "Hello World: Binary Streaming" as binary data
    writeBinaryFileWithStreaming(filePath, binaryData) // Does not append but replaces existing file or create a new file
}

main()

true

In [13]:
// Reading a binary file with streaming
fun main() = runBlocking {
    val folderName = "files"
    val fileName = "write-read-binary-streaming"
    val filePath = buildWDPath(folderName, fileName)
    
    val output = readBinaryFileWithStreaming(filePath)
    output?.let { it ->
        // Convert binary data back to text using UTF-8 encoding
        // We already know data is UTF_8 text
        val textRead = String(it, Charsets.UTF_8)
        println(textRead)
    }
}

main()

Hello World: Binary Streaming


In [14]:
// Removing the file
fun main() = runBlocking {
    val folderName = "files"
    val fileName = "write-read-binary-streaming"
    val filePath = buildWDPath(folderName, fileName)
    
    withContext(Dispatchers.IO) {
        try {
            File(filePath).delete()
            println("File ${filePath} was removed.")
        } catch (e: IOException) {
            println("Error removing file: ${e.message}")
        }
    }
}

main()

File .\files\write-read-binary-streaming was removed.
